In [148]:
import pandas as pd

data = pd.read_csv('data_dev.txt', sep='~', engine='python', index_col=False)
company_map = pd.read_csv('company_map.txt')

In [149]:
import re
p = re.compile(r"[^a-zA-Z0-9!&'()*/ .-]")
def remain_vaild_char(string):
    tmp_string = re.sub('\n', ' ', string)
    tmp_string = re.sub(p, '', tmp_string)
    return tmp_string

data['refine_headline'] = data['Headline'].apply(remain_vaild_char)
data['refine_body'] = data['Body'].apply(remain_vaild_char)

In [150]:
company_map['split_officialname'] = company_map['OfficialName'].apply(lambda x : x.split())

In [151]:
data['split_body'] = data['refine_body'].apply(lambda x : x.split())
data['split_head'] = data['refine_headline'].apply(lambda x : x.split())

In [152]:
from itertools import chain

name_partial_list = list(chain.from_iterable(company_map['split_officialname'].values))
body_partial_list = list(chain.from_iterable(data['split_body'].values))
head_partial_list = list(chain.from_iterable(data['split_head'].values))

In [153]:
text_list = body_partial_list+head_partial_list

In [154]:
from collections import Counter
counter1 = Counter(name_partial_list)
counter2 = Counter(text_list)

In [206]:
company_high_frequency_word = [pair[0] for pair in counter1.items() if pair[1] >= 10]

In [218]:
text_high_frequency_word = [pair[0] for pair in counter2.items() if pair[1] >= 30]

In [219]:
temp=[]
for word_list in company_map.split_officialname:
    tmp=[]
    for word in word_list:
        if word in company_high_frequency_word:
            if len(word_list) <= 2:
                tmp.append(word)
        if word not in company_high_frequency_word:
            tmp.append(word)
    temp.append(tmp)       

In [220]:
company_map['refine_officialname'] = company_map['split_officialname'].apply(lambda x : [word for word in x if word not in company_high_frequency_word])
company_map['refine_officialname'] = company_map['refine_officialname'].apply(lambda x : [word for word in x if word not in text_high_frequency_word])

In [221]:
company_map['refine_officialname_add'] = company_map['refine_officialname'].apply(lambda x : ' '.join(x))

In [222]:
word_dict = {}
empty_list = []
for _, row in company_map.iterrows():
    tmp_add = word_dict.get(row['refine_officialname_add'],[])
    tmp_add.append(row['CompanyID'])
    word_dict[row['refine_officialname_add']] = tmp_add
    for word in row['refine_officialname']:
        tmp = word_dict.get(word, [])
        tmp.append(row['CompanyID'])
        word_dict[word] = tmp

In [223]:
import re
p = re.compile(r"[^a-zA-Z0-9!&'()*/ .-]")
def remain_vaild_char(string):
    tmp_string = re.sub('\n', ' ', string)
    tmp_string = re.sub(p, '', tmp_string)
    return tmp_string

data['refine_headline'] = data['Headline'].apply(remain_vaild_char)
data['refine_body'] = data['Body'].apply(remain_vaild_char)

In [224]:
def predict(string):
    tmp = [word_dict.get(word.upper()) for word in string.split() if word_dict.get(word.upper())]
    tmp = list(chain.from_iterable(tmp))
    c_tmp = Counter(tmp)
    print(c_tmp.most_common()[:3])
    return [item[0] for item in c_tmp.most_common()[:3] if item[1] != 1]

In [225]:
#data["TEXT"] = data['refine_body'] + data['refine_headline']

In [226]:
data['pred'] = data['refine_headline'].apply(predict)
#data['pred'] = data['refine_body'].apply(predict)
#data['pred'] = data['TEXT'].apply(predict)

[(2810, 1), (1536, 1)]
[(3430, 2), (2408, 2), (2810, 1)]
[(1747, 2), (789, 2), (2810, 1)]
[(3330, 2), (2810, 1), (1188, 1)]
[(1844, 3), (3253, 2), (2171, 1)]
[(3166, 2)]
[(849, 2), (291, 2), (1995, 1)]
[(1117, 2), (2810, 1)]
[(359, 2), (1314, 1)]
[(2012, 2), (2810, 1)]
[(2810, 1), (739, 1), (3721, 1)]
[(3315, 2), (2810, 1), (260, 1)]
[(991, 2), (1821, 2), (1619, 2)]
[(3581, 2), (2810, 1), (260, 1)]
[(1614, 2), (2810, 1), (3466, 1)]
[(2494, 2), (2540, 2), (2810, 1)]
[(2810, 1), (2039, 1), (2704, 1)]
[(20, 2), (2810, 1)]
[(1066, 1)]
[(2810, 1), (1188, 1), (2675, 1)]
[(1049, 1), (950, 1), (93, 1)]
[(279, 2), (2810, 1)]
[(2810, 1), (620, 1), (260, 1)]
[(3165, 2), (1314, 1)]
[(1284, 2), (2810, 1), (260, 1)]
[(577, 2), (1269, 2), (3520, 1)]
[(16, 2), (171, 2), (2518, 1)]
[(2284, 2), (2810, 1)]
[(254, 2)]
[(3356, 2), (2810, 1), (54, 1)]
[(2179, 2), (2078, 2), (2810, 1)]
[(2120, 2), (2810, 1)]
[(3337, 2), (2810, 1), (1704, 1)]
[(2810, 1), (3329, 1), (660, 1)]
[(1364, 2), (3448, 2), (1340, 1)]


In [216]:
unique_company = company_map.CompanyID.values

## Jaccard similarity

In [217]:
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.metrics import jaccard_similarity_score, hamming_loss

def Onehot(CompanyID, CompanyID_pred, unique):
    
    company_id = [i.split(",") for i in CompanyID]
    temp=[]; tmp=[];
    for i in company_id:
        k=[]
        for j in i:
            k.append(j.split(" "))
        temp.append(k)
    for i in temp:
        kk=[]
        for j in i:
            for l in j:
                kk.append(int(l))
        tmp.append(kk)
    pred = list(CompanyID_pred)

    mlb = MultiLabelBinarizer(classes=unique)
    onehot = mlb.fit_transform(tmp)
    onehot_pred = mlb.fit_transform(pred)
    
    return onehot, onehot_pred

onehot, pred = Onehot(data.CompanyID,data.pred,unique_company)
jaccard_similarity_score(onehot, pred)

0.5233261904761906